In [ ]:
! apt-get install git
! git clone https://github.com/joaorossi15/requirements-to-us.git

In [ ]:
! pip install datasets
! pip install optimum
! pip install auto-gptq
! pip install bitsandbytes

In [3]:
import transformers
import peft
import pandas as pd
import datasets
import torch
import gc
import re
import os

os.environ["WANDB_DISABLED"] = "true"
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device
gc.collect()

30

In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
def load_model(model_name: str):
    m = transformers.AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map='cuda',
        trust_remote_code=False,
        revision='main',
    )

    return m.to(device)


model_name = "mistralai/Mistral-7B-Instruct-v0.3"
m = load_model(model_name=model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [6]:
def construct_instruction() -> str:
    transparency = "Transparency: Transparency can typically be understood in two ways: the transparency of the AI technology itself and the transparency of the AI organisations developing and using it. Throughout our analysis, transparency was regularly discussed directly, or in relation to processes required to ensure it, such as explainability, understandability and communication."
    n_malef = "Non-maleficence: The principle of non-maleficence gained attention, and in its most basic form, it means to do no harm or avoid doing harm to others"
    resp = "Responsibility: The principle of responsibility is base on responsibility of the developers and stakeholders over the AI, accountability, liability and acting with integrity"
    privacy = "Privacy: Related to privacy of personal data, because of the large abundance of data that is required for AI to work, it is important that individuals privacy is not jeopardised as a result"
    benefiecnce = "Beneficence: Beneficence essentially means to do good, to carry out an activity with the intention of benefitting someone or society as a whole"
    f_and_a = "Freedom and autonomy: Democratic societies place value in freedom and autonomy, and it is important that AI use does not encumber or harm these for us"
    sus = "Sustainability: All fields and disciplines are affected and need to incorporate sustainability agendas"
    dig = "Dignity: Human dignity is the recognition that individuals have inherent worth and that their rights should be respected"
    j_and_f = "Justice and fairness: The issue of discrimination and unfair results resulting from algorithms has become a significant concern. It is imperative that systems are designed to ensure that they are free from any form of unfairness and inequality."
    trust = "Trust: Trust is built by keeping promises, making sure systems work properly and protecting data responsibly. Organisations must prove their trustworthiness by ensuring that their technologies are secure and effective."
    instruction = f'You are an ethical requirements engineer translating requirements to ethical user stories based on one of the 10 ethical principles: {transparency}\n {n_malef}\n {resp}\n {privacy}\n {benefiecnce}\n {f_and_a}\n {sus}\n {dig}\n {j_and_f}\n {trust}\n and the TEMPLATE:\n Title: <title>\n Description: As a <persona> i want to <do something> <so that>\n Work: <acceptance criteria>\n Now following this template and the ethical principles definition, choose only one of the ethical principles and transform the requirement into an ethical user story based on that principle, containing a Title, substituting <title> for a descriptive user story title, Description of an ethical user story, substituting <persona> for the one that requires this functionality, <do something> for what the <persona> wants to do and <so that> to the end goal of the functionality and Work, substituting <acceptance criteria> with a checklist of small tasks that need to be done so that the user story can be considered done, also called acceptance criteria'
    instruction = instruction.lower()

    return instruction

def text_to_lower(text: str) -> str:
    return text.lower()

def pad_multiple_spaces(text: str) -> str:
    return re.sub( r"\s+"," ",text)

def apply_in_row(row, instruction):
    return f'''[INST] {instruction} \n {row['text']} \n [/INST] {row['ethical_us']}</s>'''



In [7]:
def load_dataset():
    df = pd.read_csv('./requirements-to-us/data/dataset-ethical-ai.csv')
    df = df[['text', 'is_major_principle_here', 'ethical_us']]

    return df

def prepare_dataset(df: pd.DataFrame) -> pd.DataFrame:
    df['text'] = df['text'].apply(text_to_lower)
    df['ethical_us'] = df['ethical_us'].apply(text_to_lower)
    df['text'] = df['text'].apply(pad_multiple_spaces)
    df['ethical_us'] = df['ethical_us'].apply(pad_multiple_spaces)
    instruction = construct_instruction()
    df['data'] = df.apply((lambda row: apply_in_row(row, instruction)), axis=1)

    return df

def tokenize_dataset(model_name: str):
    t = transformers.AutoTokenizer.from_pretrained(model_name, use_fast=True)
    t.pad_token = t.eos_token
    data_collator = transformers.DataCollatorForLanguageModeling(t, mlm=False)

    return data_collator, t

def train_test_split(df: pd.DataFrame) -> pd.DataFrame:
    df = datasets.Dataset.from_pandas(df)
    df = df.train_test_split(test_size=0.2)

    return df


In [8]:
df = load_dataset()
df = prepare_dataset(df)
data_collator, t = tokenize_dataset(model_name)
data = train_test_split(df)

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [9]:
def tokenize_function(df):
    # extract text
    text = df["data"]

    #tokenize and truncate text
    t.truncation_side = "left"
    tokenized_inputs = t(
        text,
        return_tensors="np",
        truncation=True,
        max_length=512
    )

    return tokenized_inputs

tokenized_data = data.map(tokenize_function, batched=True)

Map:   0%|          | 0/874 [00:00<?, ? examples/s]

Map:   0%|          | 0/219 [00:00<?, ? examples/s]

In [10]:
def train_model(model, lr, batch_size, num_epochs, tokenized_data, collator):
    model.train() # training state
    model.gradient_checkpointing_enable()
    model = peft.prepare_model_for_kbit_training(model) # turn into qlora

    # lora config
    config = peft.LoraConfig(
        r=32,
        lora_alpha=64,
        target_modules=["q_proj"],
        lora_dropout=.1,
        bias="none",
        task_type="CAUSAL_LM"
    )
    config.inference_mode = False

    model = peft.get_peft_model(model, config) # model in lora style

    training_args = transformers.TrainingArguments(
        output_dir= "checkpoints_output",
        learning_rate=lr,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs,
        weight_decay=0.01,
        logging_strategy="epoch",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        warmup_steps=2,
        fp16=True,
        optim="paged_adamw_8bit",
        report_to=None
    )

    trainer = transformers.Trainer(
        model=model,
        train_dataset=tokenized_data["train"],
        eval_dataset=tokenized_data["test"],
        args=training_args,
        data_collator=collator,
    )

    # train model
    model.config.use_cache = False
    trainer.train()

    # renable warnings
    model.config.use_cache = True

    return model, trainer

final_model, trainer = train_model(m, 1e-4, 4, 3, tokenized_data, data_collator)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss
1,0.742900,0.543902
2,0.491900,0.512037
3,0.454600,0.506296


In [11]:
prompt_template = lambda comment: f'''[INST] Transform the requirement into an ethical user story following the TEMPLATE, substituting whats inside the <>:\n Title: <title>\n Description: As a <persona> i want to <do something> <so that>\n Work: <acceptance criteria>: {comment} [/INST]'''
comment = "The user need an authentication token to login."
prompt = prompt_template(comment)

final_model.eval()
inputs = t(prompt, return_tensors="pt")
outputs = final_model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=95)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [12]:
print(t.batch_decode(outputs)[0].split('[/INST] ')[1])

Title: Secure User Authentication

Description: As a security-conscious user, i want to be required to have an authentication token to login so that my account and personal data remain protected from unauthorized access.

Work:
- The system must prompt the user for their login credentials (username and password).
- Upon successful verification of the login credentials, the system must generate a unique authentication token.
- The system must require the user to enter the


In [13]:
final_model.save_pretrained("./requirements-to-us/trained-model")

In [14]:
final_model.push_to_hub("joaorossi15/mistral-7B-v03-ethical-us")

adapter_model.safetensors:   0%|          | 0.00/33.6M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/joaorossi15/mistral-7B-v03-ethical-us/commit/307da9d2d45e94169669c361b219c38872df8e83', commit_message='Upload model', commit_description='', oid='307da9d2d45e94169669c361b219c38872df8e83', pr_url=None, repo_url=RepoUrl('https://huggingface.co/joaorossi15/mistral-7B-v03-ethical-us', endpoint='https://huggingface.co', repo_type='model', repo_id='joaorossi15/mistral-7B-v03-ethical-us'), pr_revision=None, pr_num=None)